## Assignment 2
In this exercise you will:
 * Compute an implicit MLS function approximating a 3D point cloud with given (but possibly unnormalized) normals.
 * Sample the implicit function on a 3D volumetric grid.
 * Apply the marching tets algorithm to extract a triangle mesh of this zero level set.
 * Experiment with various MLS reconstruction parameters.

Your main task is to construct an implicit function `f(x)` defined on all `x`  R<sup>3</sup> whose zero level set contains (or at least passes near) each input point. That is, for every point `pi` in the point cloud, we want `f(pi) = 0`. Furthermore, `∂f` (the isosurface normal) evaluated at each point cloud location should approximate the point's normal provided as input.

In [4]:
import numpy as np
import igl
import meshplot as mp
import math
import matplotlib.pyplot as plt

In [5]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners
# returns V - vertices of the grid

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [6]:
def find_closest_point(query_point, point_set):
    dist = np.linalg.norm(point_set - query_point, axis=1)
    return np.argmin(dist)

In [7]:
"""
pp = p+ 
npp = p- 
"""
def setupConstraints(pis, v, eps, ps):
    ni = igl.per_vertex_normals(pis, v)  
    f = np.zeros(len(pis) * 3)
    pps = np.zeros_like(pis)  # positive points
    nps = np.zeros_like(pis)  # negative points
    # compute p+
    for idx, pi in enumerate(pis):
        pp = pi + eps * ni[idx]
        cp = find_closest_point(pp, pis)
        new_eps = eps
        while cp != idx:
            new_eps /= 2  # Reduce step size
            pp = pi + new_eps * ni[idx]
            print("not closest point!")
            cp = find_closest_point(pp, pis)
        pps[idx]=pp
        f[len(pis) + idx] = new_eps
    # compute p-
    for idx, pi in enumerate(pis):
        npp = pi - eps * ni[idx]  # Negative offset
        cp = find_closest_point(npp, pis)
        new_eps = eps
        while cp != idx:
            new_eps /= 2
            npp = pi - new_eps * ni[idx]
            print("not closest point!")
            cp = find_closest_point(npp, pis)
        nps[idx]=npp
        f[2*len(pis) + idx] = -new_eps
    p=np.concatenate((pis, pps, nps), axis=0)
    print("length of p = 3n: ", len(p) == (len(pis)+len(pps)+len(nps))) #check to make sure p is correct
    print("length of f = 3n: ", len(f) == (len(pis)*3)) #check to make sure f is correct
    # plot 3 set of points
    plot = mp.plot(pis, shading={"point_color": "blue", "point_size": ps}) # blue = on surface
    plot.add_points(pps, shading={"point_color": "red", "point_size": ps}) # red = outside 
    plot.add_points(nps, shading={"point_color": "green", "point_size": ps}) # green = inside 
    return p,f

### Create a grid sampling the 3D space
Create a regular volumetric grid around your point cloud: compute the axis-aligned bounding box of the point cloud, enlarge it slightly, and divide it into uniform cells (tets). The grid resolution is configured by
the global variable `resolution`, which can be changed. *Note* the funciton to generate the grid is provided. We call the grid vertices `x` and the tets connecting them `T`.

In [8]:
minimum = -float('inf') # temporary value to help us compute the bounding box 
maximum = float('inf')
resolution = 0.05 # we multiply the box diagonal by this when making the tetrahedral gird

#function to get margins for bounding box
def boundingBox(pis):
    min_values = np.min(pis, axis=0)
    max_values = np.max(pis, axis=0)
    return min_values, max_values

def create3dGrid(pis, n, ps_grid, ps_mesh,showCheck=False, resolution=0.05):
    bbox_min, bbox_max = boundingBox(pis) # compute axis-aligned bounding box of the point cloud 
    print(bbox_min, bbox_max)
    print("length of cube: ", (bbox_max[0]-bbox_min[0])/n)
    box_diag = np.linalg.norm(bbox_max - bbox_min)
    x,T = tet_grid((n,n,n), bbox_min-resolution*box_diag, bbox_max + resolution * box_diag)
    if(showCheck):# show if object fits inside grid
        plot = mp.plot(x, shading={"point_color": "black",  "point_size": ps_grid})
        plot.add_points(pis, shading={"point_color": "blue", "point_size": ps_mesh})
    return x,T

### MLS Interpolation
We now use MLS interpolation to construct an implicit function satisfying the constraints as nearly as possible.
We won't define the function with an explicit formula; instead we characterize it as the linear combination of polynomial basis functions that best satisfies the constraints in some sense. At a given point `xi` in `x`, you evaluate this function by finding the "optimal" basis function coefficients (which will vary from point to point!) and using these to combine the basis function values at `xi`.

Complete the appropriate source code sections to evaluate the MLS function at every node `xi` of a regular volumetric
grid containing the input point cloud. As an example, the provided code computes the grid values for an implicit function representing a sphere (MLS wasn't used in
this case since the formula is known analytically).


More specifically, for each grid node `xi` of the grid, evaluate the implicit function `f(xi)`, whose zero level set approximates the point cloud.
Use the moving least squares approximation presented in class and in the tutoring session. You should use the
Wendland weight function with radius configured by `wendlandRadius` and degree `k = 0, 1, 2` polynomial basis functions configured by `polyDegree`. Only use the constraint points with nonzero weight (i.e., points `p` with `||xi - p|| < wendlandRadius`). *Note* if the number of constraint points within `wendlandRadius` is less than twice the number of polynomial coefficients (i.e., 1 for `k = 0`, 4 for `k = 1`, and 10 for `k = 2`), you can assign a large positive (outside) value to the grid point.

Store the field value  `fx = f(xi)` in a  `numpy.array`, using the same ordering as in `x`. Render these values by coloring each grid point red/green depending if they are inside outside (i.e., `fx < 0` or `fx ≥ 0`). You can use the `meshplot.plot(..., c=color)` where `color` is a `n x 3` matrix containing rgb values.

**Important**: explicitty write a function `closest_points(point, points, h)` that retreives the indices all points in `points` that are at distance less than `h` from `point`.

*Relevant `numpy` functions:* `argwhere`, `linalg.solve`.

Required output of this section:

 * Plot of the grid points `x` colored according of being inside or outside the input cloud.


In [33]:
"""
Goal: find the best f in the span of chosen basis functions 
    f(x) = sum_j b_j(x)a_j, where the basis is a polynomial function 
"""

def check_rank(matrix):
    rank = np.linalg.matrix_rank(matrix)
    print("Rank:", rank, "Size:", matrix.shape[0])
    return rank < matrix.shape[0]
""" 
Compute polynomial basis (at a point) in 2nd degree

1,x,y,z,x^2,y^2,z^2,xy,xz,yz
""" 
def polynomialBasis(point):
    x,y,z = point;
    return np.array([1,x,y,z,x**2,y**2,z**2,x*y,x*z,y*z]) #return np.array([1,x,y,z])

def polynomialBasisMat(x):
    B = np.apply_along_axis(polynomialBasis, axis=1, arr=x)
    return B
def mls_interpolate(p,f, x,T, wendlandRadius):
    # We want to solve (B^TW(x)B)a(x)=B^TW(x)d, where W = wendland weight matrix 
    # check to make sure p and x have same dimensionality 
    assert p.shape[1] == x.shape[1]
    # 1. Compute B (the polynomial basis of p) -- polynomial 3d basis = 1,x,y,z,x^2,y^2,z^2,xy,xz,yz
    B = polynomialBasisMat(p)
    print("Shape of B: ", B.shape)

    # 2. Compute W  -- for each evaluation point, evaluate pi, in p, W_ii = w||x-p_i||
    ##  x[:, None, :] has shape (k, 1, d)
    ##  p[None, :, :] has shape (1, 3n, d)
    ##  resulting distances have shape (k, 3n)
    distances = np.linalg.norm(x[:, None, :] - p[None, :, :], axis=2) # using numpy broadcasting
    # print("Distances shape:", distances.shape)
    weights = wendland(distances, wendlandRadius) # getting the wendland weights for all distances at once
    print(weights)
    # For each point x compute a, and then fx
    fx = np.zeros(len(x))
    for i in range(len(x)):
        W_i = np.diag(weights[i]) # computing wendland matrix 
        # print("Wi shape:", W_i.shape)
        BTW = B.T @ W_i
        lhs = BTW @ f
        if check_rank(lhs):
            print("lhs is singular or rank-deficient! in round ", i, " of ", len(x))
            print(lhs)
        rhs = BTW @ B
        if check_rank(rhs):
            print("rhs is singular or rank-deficient! in round ", i, " of ", len(x))
            print(rhs)
        coeff = np.linalg.solve(rhs, lhs)
        # now we have to compute f(x) at every point for every a 
        basis_at_x_i = polynomialBasis(x[i])
        fx[i] = np.dot(basis_at_x_i, coeff)
    return fx    

def mls_interpolate_with_close_points(p,f, x,T, closestPointDist, wendlandRadius):
    # We want to solve (B^TW(x)B)a(x)=B^TW(x)d, where W = wendland weight matrix 
    # check to make sure p and x have same dimensionality 
    assert p.shape[1] == x.shape[1]
    fx = np.zeros(len(x))
    for i in range(len(x)):
        # get the indices closest points to x
        indices = closest_points(x[i], p, closestPointDist)
        if len(indices) <= 0:
            continue  # if no close points are found # print("num closest points: ", len(indices))
        # get the actual closest points to x
        p_closest = p[indices]  
        f_closest = f[indices]
        # compute B
        B = polynomialBasisMat(p_closest)
        # get the weights and wendland matrix 
        distances = np.linalg.norm(x[i] - p_closest, axis=1)  
        weights = wendland(distances, wendlandRadius)
        W_i = np.diag(weights) # computing wendland matrix 
        # solve for coefficients (a) 
        BTW = B.T @ W_i
        lhs = BTW @ f_closest
        rhs = BTW @ B
        coeff = np.linalg.solve(rhs, lhs)
        # compute f(x) = b(x)a(x)
        basis_at_x_i = polynomialBasis(x[i])
        fx[i] = np.dot(basis_at_x_i, coeff)
    return fx
"""
returns the indices of the points within distance h of point, point
"""
def closest_points(point, points, h):
    dist = np.linalg.norm(points - point, axis=1)
    return np.where(dist < h)[0]

"""
Computes wendland weight 
r is the Euclidean distance from the evaluation point.
h (the Wendland radius) defines the range of influence.
"""
def wendland(r, h):
    return np.maximum(0,(1-r/h)**4*(4*r/h+1)) 

In [24]:
def calculate_wendland_radius(p, num_neighbors=10):
    # p is the point cloud of shape (n_points, 3)
    n_points = p.shape[0]
    # Step 1: Calculate pairwise distances between all points
    dist_matrix = np.linalg.norm(p[:, np.newaxis] - p, axis=2)  # Shape (n_points, n_points)
    # Step 2: For each point, sort the distances to other points
    # We don't need the distance to itself, so we ignore the diagonal (i.e., i==j)
    sorted_distances = np.sort(dist_matrix, axis=1)[:, 1:]  # Ignore self distance (diagonal)
    # Step 3: For each point, take the distance to the 10th nearest neighbor
    distances_to_10th = sorted_distances[:, num_neighbors-1]
    # Step 4: Choose the Wendland radius (take average of the 10th nearest distances)
    wendland_radius = np.mean(distances_to_10th)
    return wendland_radius

In [25]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
wendland_radius = calculate_wendland_radius(pi)
print(f"Calculated Wendland radius: {wendland_radius}")

Calculated Wendland radius: 11.798686063738305


In [31]:
## Reading in the mesh
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
# mp.plot(pi, shading={"point_size": 8})
## Setting constraints
eps = 1
ps = 5
p,f=setupConstraints(pi, v, eps, ps)
## Adding the Grid def create3dGrid(pis, n, ps_grid, ps_mesh,showCheck=False, resolution=0.05):
x, T = create3dGrid(p, 20, 2,8,True) # was 80 | 1 |  5
# print(x.shape)
## Running the interpolation 
fx = mls_interpolate(p,f, x,T, 100, 11)
## Running marching tets
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0) ## SV AND SF ARE THE VERTICES
mp.plot(sv, sf, shading={"wireframe": True})

length of p = 3n:  True
length of f = 3n:  True


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

[-23.68137397 -64.69682439 -99.63522961] [33.69826224 17.66157433  6.79268375]
length of cube:  2.86898181084974


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0084438…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0084438…

In [136]:
# ## Reading in the mesh
# pi, v = igl.read_triangle_mesh("data/cube.obj")
# pi /= 10
# ni = igl.per_vertex_normals(pi, v)
# mp.plot(pi, shading={"point_size": .05})
# ## Setting constraints
# eps = .01
# ps = .05
# p,f=setupConstraints(pi, v, eps, ps)
# ## Adding the Grid def create3dGrid(pis, n, ps_grid, ps_mesh,showCheck=False, resolution=0.05):
# x, T = create3dGrid(p, 10, .025,.05,True) # was 80 | 1 |  5
# # print(x.shape)
# ## Running the interpolation 
# fx = mls_interpolate(p,f, x,T, 5, .2)
# ## Running marching tets
# sv, sf, _, _ = igl.marching_tets(x, T, fx, 0) ## SV AND SF ARE THE VERTICES
# mp.plot(sv, sf, shading={"wireframe": True})

## Implementing a spatial index to accelerate neighbor calculations

To construct the MLS equations, you will perform queries
to find, for a query point `q`:

    * the closest input point to `q` (needed while constructing inside/outside offset points); and
    * all input points within distance `h` of `q` (needed to select constraints with nonzero weight).


Although a simple loop over all points could answer these queries, it would be slow for large point clouds.
Improve the efficiency by implementing a simple spatial index (a uniform grid at some resolution). By this, we mean binning vertices into their enclosing grid cells and restricting the neighbor queries to visit only the grid cells that could possibly satisfy the query. You can debug
this data structure by ensuring that it agrees with the brute-force for loop implementation.

This part requires changing the two fucntion `find_closed_point` and `closest_points`.

## Using a non-axis-aligned grid.
The point cloud `luigi.off` is not aligned with the canonical axes.
Running reconstruction on an axis-aligned grid is wasteful in this case: many of the grid points will lie far outside the object. Devise an automatic (and general) way to align the grid to the data and implement it.

Required output of this section:

* Plot of the grid with nodes colored according to their implicit function values

## Extracting the surface
You can now use marching tets to extract the zero isosurface from your grid.
The extraction has already been implemented and the surface is displayed. The implicit function obtained from MLS might be noisy and the reconstructed mesh will contain several pieces. Filter out and keep the largest component.

![](img/cat-recon.png?raw=true)

*Relevant `igl` functions:* `marching_tets`, `face_components`.


Required output of this section:

* Plot of the reconstructed surfaces. Experiment with different parameter settings: grid resolution (also anisotropic in the 3 axes), Wendland function radius, and polynomial degree.


## MLS function

In [48]:
# Parameters
bbox_min = np.array([-1., -1., -1.])
bbox_max = np.array([1., 1., 1.])
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

n = 10

In [51]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function

center = np.array([0., 0., 0.])
radius = 1
fx = np.linalg.norm(x-center, axis=1) - radius ## THIS IS WHERE WE WILL PUT IN THE MLS 
print(x.shape)
# print(x-center)
print(fx.shape)

(1000, 3)
(1000,)


In [81]:
# Threshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] = 1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 0.1,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Marching tets to extract surface

In [90]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0) ## SV AND SF ARE THE VERTICES
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Optional tasks

<!-- * *(2 points)* Compute the closed-form gradient of the MLS approximation. Suggestion: A good strategy to solve this exercise is to write MLS explicitly in matrix form and then compute its gradient (a good reference for differentiating expressions with matrices can be found in [The Matrix Cookbook](http://orion.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf).
 -->
* *(2 points)* In [Interpolating and Approximating Implicit Surfaces from Polygon Soup](http://graphics.berkeley.edu/papers/Shen-IAI-2004-08/index.html) normals are used differently to define the implicit surface. Instead of generating new sample points offset in the positive and negative normal directions, the paper uses the normal to define a linear function for each point cloud point: the signed distance to the tangent plane at the point.
Then the values of these linear functions are interpolated by MLS. Implement Section 3.3 of the paper and append to your report a description of the method and how it compares to the original point-value-based approach.
Estimate a normal for results obtained with single dataset.

* *(1 points)* [Screened Poisson Surface Reconstruction](http://www.cs.jhu.edu/~misha/MyPapers/ToG13.pdf) is a more modern technique that avoids some of the pitfalls of local reconstruction methods. An implementation is provided in [MeshLab](http://www.meshlab.net). A standalone implementation of this method is also provided by the authors [here](http://www.cs.jhu.edu/~misha/Code/PoissonRecon/Version9.01/) with accompanying usage instructions and datasets. Compare your MLS        reconstruction results to the surfaces obtained with this method, and try to understand the differences. Report your findings.